---
Title: Compile SQLCipher Using MSVC
Date: 2025-09-10 12:00
Lang: zh-cn
Category: SQLCipher
---

# 介绍

记录使用 `MSVC` 编译器编译 `SQLCipher` 这个加密`SQLite`扩展项目的笔记

## 准备

- `ActiveTcl`: 一个Tcl/Tk 发行版, 开箱即用(SQLCipher需要tclsh用于生产源代码和测试)
- `OpenSSL`: SQLCipher使用的加密后端

> 参考文档:
- [sqlcipher官方仓库](https://github.com/sqlcipher/sqlcipher)
- [sqlcipher社区](https://discuss.zetetic.net/c/sqlcipher/5?page=3)
- [ActiveTcl下载](https://platform.activestate.com/ActiveState/ActiveTcl-8.6)

## 编译

以`4.1.0`版本`SQLCipher`为例

- 准备 `openssl 1.1.x` 库, `ActiveTcl`包(`sqlcipher`使用`tcl`生成部分源码和测试)
- 拉取 `sqlcipher` 源码并切换到4.1.0
- 修改 `Makefile.msc` 添加`TCC`/`RCC`/`LTLINKOPTS`的额外选项, 如下所示
```makefile
TCC = $(TCC) -DSQLITE_HAS_CODEC -DSQLITE_TEMP_STORE=2 -IC:\opt\openssl-1.1.1d-x64\include
RCC = $(RCC) -DSQLITE_HAS_CODEC -DSQLITE_TEMP_STORE=2 -IC:\opt\openssl-1.1.1d-x64\include
LTLINKOPTS = $(LTLINKOPTS) /LIBPATH:C:\opt\openssl-1.1.1d-x64\lib libcrypto.lib
```
> `nmake -f Makefile.msc` 编译所有目标

**注意** 
- 不同sqlcipher版本支持的openssl-api不一致, 可根据changelog查看对应支持
- `4.1.0` 版本只需要开启 `SQLITE_HAS_CODEC` 宏即可


## 示例

- `sqlite3 :memory: "PRAGMA cipher_version;"` 查看sqlcipher版本
- `sqlite3 :memory: "SELECT sqlite_version();"` 查看对应的sqlite版本
- `sqlite3 :memory: "PRAGMA compile_options;"` 查看编译选项

**注意** 加解密的`key`(密码)/`cipher_compatibility`(加密版本)/`kdf_iter`(密钥迭代次数)/`cipher`(加密方法) 都必须一致, 才能正确解密


### 使用C-API操作db

```c

```


### python binding of sqlcipher

使用 `pysqlcipher3` 包完成 `sqlcipher` 的py绑定

> 参考:
- [pysqlcipher3仓库](https://github.com/rigglemania/pysqlcipher3.git)

**环境**

`MSVC 19`/`cl for x64`/`sqlcipher 4.1.0`/`openssl 1.1.1d`

**步骤**

- 准备 `sqlcipher` 的源码, 生成加密版本的 `sqlite3.c` 文件
- 拉取 `pysqlcipher3` 的代码仓库
- 复制 `sqlite3.c`/`sqlite3.h` 到 `pysqlcipher3/amalgamation` 下
- 修复 `pysqlcipher3/setup.py` 的高版本兼容问题
    > `cl` 定义python API模块名宏 `MODULE_NAME` 的字符串错误, 在 `quote_argument` 方法修改 `quote = '"' if sys.platform != 'win32' else '\\"'` 为 `quote = '"' if sys.platform != 'win32' else '\"'`
    > `OPENSSL_CONF` 在 `1.1` 以下版本中存在, `pysqlcipher3` 仅用它来定位库路径, `1.1.1d` 版本中可以使用`libcrypto.lib`的绝对路径来代替
    > 链接库 `libeay32.lib` 在 `1.1.0` 版本中被弃用, 更改为 `libcrypto.lib`
- 设置 `OPENSSL_CONF` 环境变量及激活msvc_x64终端
- 使用 `python setup.py build_amalgamation` 的源码编译版本, 而不是 `build_ext` 的库链接版本

#### 使用pysqlcipher3操作sqlite3

```py
from pysqlcipher3 import dbapi2 as sqlite
conn = sqlite.connect('test.db')
c = conn.cursor()
c.execute("PRAGMA key='password'")
c.execute("PRAGMA cipher_compatibility = 3")
c.execute('''create table if not exists stocks (date text, trans text, symbol text, qty real, price real)''')
c.execute("""insert into stocks values ('2006-01-05','BUY','RHAT',100,35.14)""")
c.execute("""select * from stocks""")
print(c.fetchall())
conn.commit()
c.close()
```